In [21]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader


In [ ]:
loader_info_txt = DirectoryLoader(
    "/home/austin/RAG/data/info",
    glob="**/*.txt"
)

doc_info = loader_info_txt.load()
print(doc_info)

loader_pg_pdf = DirectoryLoader(
    "/home/austin/RAG/data/pg",
    glob="**/*.pdf"
)
doc_pg = loader_pg_pdf.load()
print(doc_pg)

loader_ug_syllabus= DirectoryLoader(
    "/home/austin/RAG/data/ug/syllabus",
    glob="**/*.pdf"
)

doc_ug_syllabus=loader_ug_syllabus.load()
print(doc_ug_syllabus)

loader_ug_prospect= DirectoryLoader(
    "/home/austin/RAG/data/ug/prospect",
    glob="**/*.pdf"
)

doc_ug_prospect =loader_ug_prospect.load()
print(doc_ug_prospect)

doc_ug = doc_ug_prospect + doc_ug_syllabus


[Document(metadata={'source': '/home/austin/RAG/data/info/FAQ.txt'}, page_content='FAQ [Frequently Asked Questions]\n\n1. What are the mandatory steps that I have to follow to become a valid applicant?\n\nA. The first step of the CSS online registration process is to click on the "Click here for Registration" button available on the homepage of the website www.admissions.keralauniversity.ac.in. After that, the candidate has to provide basic details such as Name, Date of Birth, Gender, Email ID, and Mobile Number. Once the required details are entered, an OTP will be sent for mobile number verification. After successful verification, a unique Application Number will be provided.\n\nB. After generating the Application number the candidate should login with the Application number and Password. Date of Birth is the password by default (DD/MM/YYYY)\n\nC. The candidate should follow the 5 steps of the online registration process as mentioned in the prospectus.\n\nD. DO NOT SEND HARD COPIES O

In [38]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [46]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=50,
    length_function=len,
    separators=["\n\n", "\n", ".", "!", "?", " ", ""]
)

chunks_ug = text_splitter.split_documents(doc_ug)
chunks_pg = text_splitter.split_documents(doc_pg)
chunks_info = text_splitter.split_documents(doc_info)

chunk_groups = [
    ("UG Documents", chunks_ug),
    ("PG Documents", chunks_pg),
    ("Info Documents", chunks_info)
]

for name, chunks in chunk_groups:
    print(f"{name}: Split into {len(chunks)} chunks.")


UG Documents: Split into 418 chunks.
PG Documents: Split into 237 chunks.
Info Documents: Split into 12 chunks.


In [54]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3',  
                       use_fp16=True)

Fetching 30 files: 100%|██████████| 30/30 [01:00<00:00,  2.01s/it]


In [56]:
sentences_1 = ["What is BGE M3?", "Defination of BM25"]
sentences_2 = ["BGE M3 is an embedding model supporting dense retrieval, lexical matching and multi-vector interaction.", 
               "BM25 is a bag-of-words retrieval function that ranks a set of documents based on the query terms appearing in each document"]



embeddings_1 = model.encode(sentences_1, 
                            batch_size=12, 
                            max_length=1000, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
                            )['dense_vecs']
embeddings_2 = model.encode(sentences_2)['dense_vecs']
similarity = embeddings_1 @ embeddings_2.T
print(similarity)
# [[0.6265, 0.3477], [0.3499, 0.678 ]]

[[0.6259036  0.3474959 ]
 [0.34986806 0.6782464 ]]
